In [1]:
import pandas as pd
import requests
import json # Importa el módulo 'json'

In [ ]:
def obtener_datos(series_id, site_code):
    """
    Realiza una petición a la API y devuelve un DataFrame.
    """
    url = f"https://alerta.ina.gob.ar/pub/datos/datos&timeStart=2025-09-18&timeEnd=2025-09-20&seriesId={series_id}&siteCode={site_code}&varId=2&format=csv"
    
    try:
        # Lee el CSV directamente desde la URL.
        df = pd.read_csv(url, comment='#', sep=',', header=None)
        df.columns = ['obs_id', 'timestart', 'timeend', 'valor', 'timeupdate']
        
        # Agrega las columnas adicionales
        df['seriesId'] = series_id
        df['siteCode'] = site_code
        
        return df
    
    except Exception as e:
        print(f"Error al obtener datos para seriesId {series_id}: {e}")
        return pd.DataFrame() # Devuelve un DataFrame vacío en caso de error

# Leer el archivo JSON de forma síncrona con Python
try:
    with open("alturasHidrometricas.json", 'r', encoding='utf-8') as file:
        alturas_json = json.load(file)
except FileNotFoundError:
    print("Error: El archivo 'alturasHidrometricas.json' no se encontró.")
    alturas_json = []

lista_de_dataframes = []

for item in alturas_json:
    df_temporal = obtener_datos(item['seriesid'], item['sitecode'])
    lista_de_dataframes.append(df_temporal)

# Concatena todos los DataFrames en uno solo
df_final = pd.concat(lista_de_dataframes, ignore_index=True)

# Muestra las primeras 5 filas del DataFrame final
print(df_final.head())

# Opcional: Guarda el DataFrame final en un solo archivo CSV
df_final.to_csv("datos_finales.csv", index=False)

Error al obtener datos para seriesId 55: No columns to parse from file
Error al obtener datos para seriesId 151: No columns to parse from file
Error al obtener datos para seriesId 3279: No columns to parse from file
Error al obtener datos para seriesId 3280: 'utf-8' codec can't decode byte 0xf3 in position 695: invalid continuation byte
Error al obtener datos para seriesId 3309: 'utf-8' codec can't decode byte 0xe1 in position 759: invalid continuation byte
Error al obtener datos para seriesId 3312: 'utf-8' codec can't decode byte 0xe1 in position 388: invalid continuation byte
Error al obtener datos para seriesId 3314: 'utf-8' codec can't decode byte 0xe1 in position 417: invalid continuation byte
Error al obtener datos para seriesId 3345: 'utf-8' codec can't decode byte 0xf3 in position 560: invalid continuation byte
Error al obtener datos para seriesId 3347: 'utf-8' codec can't decode byte 0xf3 in position 334: invalid continuation byte
Error al obtener datos para seriesId 6060: 'ut

In [2]:
df_nuevito = pd.read_csv("datos_finales.csv")
df_nuevito

,obs_id,timestart,timeend,valor,timeupdate,seriesId,siteCode
0,25671322243,2025-09-18 00:00:00,2025-09-18 00:00:00,0.66,2025-09-18 18:01:45.740123,8,8
1,25678429536,2025-09-19 00:00:00,2025-09-19 00:00:00,0.46,2025-09-19 12:01:56.589385,8,8
2,25671322611,2025-09-18 00:00:00,2025-09-18 00:00:00,8.60,2025-09-18 18:01:46.129692,9,9
3,25678429676,2025-09-19 00:00:00,2025-09-19 00:00:00,8.40,2025-09-19 12:01:56.830545,9,9
4,25671322635,2025-09-18 00:00:00,2025-09-18 00:00:00,9.20,2025-09-18 18:01:46.298099,10,10
...,...,...,...,...,...,...,...
10329,25675266304,2025-09-19 00:00:00,2025-09-19 00:00:00,1.48,2025-09-19 04:01:46.98231,37594,7392
10330,25676460271,2025-09-19 04:00:00,2025-09-19 04:00:00,1.45,2025-09-19 07:01:50.255738,37594,7392
10331,25677641323,2025-09-19 08:00:00,2025-09-19 08:00:00,1.42,2025-09-19 10:01:48.955388,37594,7392
10332,25679618659,2025-09-19 12:00:00,2025-09-19 12:00:00,1.40,2025-09-19 15:01:48.669583,37594,7392


In [11]:
df = df_nuevito.drop(['timestart', 'timeupdate'], axis=1)

In [24]:
df.iloc[1 + 10] 

obs_id              25678430019
timeend     2025-09-19 00:00:00
valor                      7.38
seriesId                     13
siteCode                     13
Name: 11, dtype: object

In [ ]:
i = 0
while i < (len(df) - 1):
    elemento = df.iloc[i]
    elemento_anterior = df.iloc[i - 1]
    series = elemento.seriesId
    site = elemento.siteCode
    values = []
    if (series != elemento_anterior.seriesId and site != elemento_anterior.siteCode) or i == 0:
        #Si el id del rio es el primero de ese rio... SEGUIR CON ESTO TENGO UN QUILOMBO BARBARO
        for indice, iscopia in df.iloc[i:].iterrows(): 
            if iscopia['seriesId'] == series and iscopia['siteCode'] == site:
                values.append(iscopia["valor"])
                indice = indice + 1
            break
        
    else:
        continue

    i = i + 1